In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import src.lib.modules as modules
%matplotlib inline
np.set_printoptions(precision=5)
pd.set_option("display.precision", 5,
              'display.max_rows', None,
              'display.max_columns', None,
              'display.width', None,
              'display.max_colwidth', None)

#  TODO: Change the names if needed
# Defining the columns that we are interested in.
x = 'Gap'
y = ['Axial force']  # should be a list
interested_in = y + [x]

# Defining specific parameters for the plots
DATA_PATH = modules.DATA_PATH
snsc = modules.get_color_palette(y)
sns.set_palette(snsc)

# Opening the files
dfs, files, common_name = modules.load_files_df(interested_in=interested_in)

# Merging the dfs and making the values absolute
df_merged = modules.merge_dfs(dfs, files)

# Plotting the raw data

In [ ]:
# NOTE: if this plot gets very messy, you can set sep_plots to True

fig, ax = plt.subplots(dpi=150, figsize=(10, 5))
modules.plot_xy(data=df_merged, x=x, y=y, ax=ax, fig=fig,
                log=True,
                # save_str=f"raw_{common_name}.png"
                )

# Calculating the average values
Just averaging out the data will account for the random uncertainty in the measurements, but not the systematic uncertainty. To account for the systematic uncertainty, we need to calculate the standard deviation of the data and use it to calculate the error bars.

In [ ]:
init_smoothness = 50
med_smoothness = 100
end_smoothness = 100

init_ratio = 0.25
med_ratio = 0.75

# making sure the ratios chosen are correct
modules.assert_ratios(init_ratio, med_ratio)

min_val = min({df['Gap'].min() for df in dfs})
max_val = max({df['Gap'].max() for df in dfs})
min_len = min({df.shape[0] for df in dfs})
max_len = max({df.shape[0] for df in dfs})

dist = max_val - min_val + 2
s0 = float(min_val - 1)
s1 = float(dist * init_ratio + s0)
s2 = float(dist * med_ratio + s0)
s3 = float(max_val + 2)

intervals = np.concatenate([
    np.arange(start=s0, stop=s1, step=min(init_smoothness, s1 - s0)),
    np.arange(start=s1, stop=s2, step=min(med_smoothness, s2 - s1)),
    np.arange(start=s2, stop=s3, step=min(end_smoothness, s3 - s2 - 1))
])

bins = pd.IntervalIndex.from_breaks(intervals)
final_df = modules.av_stderr(df=df_merged, bins=bins, x=x, y=y,
                             rename_x='Displacement',
                             # save_str=f"new_av_{common_name}.xlsx"
                             )

fig, ax = plt.subplots(dpi=150, figsize=(10, 5))
modules.plot_xy(data=final_df, x='Displacement', y=y, ax=ax, fig=fig,
                error=True,
                # save_str=f"average_{common_name}.png"
                )
